In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-sales-data-2023-2024/ecommerce_sales_dataset.csv



Business Intelligence Analyst Agent 

Problem (Why it matters)

Modern businesses generate massive amounts of data from multiple sources—sales, marketing, operations, finance—but extracting actionable insights is challenging. Current BI processes often suffer from:

Data fragmentation: Data is scattered across different systems (CRM, ERP, web analytics, spreadsheets).

Manual reporting: Analysts spend hours cleaning, aggregating, and visualizing data instead of focusing on insights.

Slow decision-making: Managers may not have timely, data-driven insights, leading to delayed or suboptimal decisions.

Example scenario: A retail company wants to identify trends in product sales and customer behavior, but their team struggles to consolidate sales data from online and offline channels, delaying marketing and inventory decisions.

Solution (What you’re proposing)

The BI Analyst Agent is an AI-driven assistant designed to streamline the full workflow of data analysis. It integrates with multiple data sources, automates data cleaning, performs analysis, and generates actionable reports. Key features include:

Automated data ingestion & cleaning: Consolidates data from CRM, ERP, spreadsheets, and APIs.

Smart insights generation: Uses BI techniques to uncover trends, anomalies, and predictive insights.

Custom dashboards & reporting: Creates interactive visualizations tailored to business needs.

Decision support: Provides recommendations based on the analysis (e.g., which products to stock, which campaigns to run).

Example scenario: Instead of spending days preparing reports, the retail team receives an AI-generated dashboard showing trending products, seasonal patterns, and predictive sales forecasts in real-time.

Value (Why it’s valuable)

The BI Analyst Agent delivers measurable business impact by:

Saving time: Reduces manual reporting, freeing analysts to focus on strategic work.

Improving accuracy: Minimizes errors in data consolidation and analysis.

Enabling faster decisions: Provides real-time insights, allowing managers to act quickly.

Driving revenue growth: By identifying trends and opportunities, businesses can optimize operations and marketing.

Impact statement: “Our BI Analyst Agent turns raw data into actionable insights automatically, transforming the way businesses make decisions and driving smarter, faster, and more profitable strategies.”

****
Core Concept

The central idea of this project is an AI-driven Business Intelligence (BI) Analyst Agent that automates the full workflow of data analysis for businesses. Unlike traditional BI tools, which require human intervention for data cleaning, aggregation, and visualization, this agent acts as an autonomous analytical assistant, capable of:

Pulling data from multiple sources (CRM, ERP, spreadsheets, APIs).

Cleaning, transforming, and integrating disparate datasets.

Performing advanced analytics (trend analysis, forecasting, anomaly detection).

Generating actionable insights and recommendations in an easily digestible format.

Innovation: The use of an intelligent agent makes the entire BI process autonomous, context-aware, and adaptive. The agent doesn’t just produce reports—it interprets them and prioritizes insights, enabling businesses to act proactively instead of reactively.

In [45]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [46]:
import asyncio
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from google.adk.tools import AgentTool, FunctionTool, google_search

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [47]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [48]:
import os

base_path = '/kaggle/input/'
print(os.listdir(base_path))  # lists all datasets added

['ecommerce-sales-data-2023-2024']


In [49]:
def clean_data(df):
    cleaned = df.copy()
    cleaned.columns = cleaned.columns.str.strip().str.lower().str.replace(" ", "_")
    cleaned = cleaned.drop_duplicates()
    cleaned = cleaned.fillna(cleaned.median(numeric_only=True))
    return cleaned


In [50]:
def generate_eda(df):
    eda_report = {
        "shape": df.shape,
        "columns": list(df.columns),
        "describe": df.describe().to_dict(),
        "nulls": df.isnull().sum().to_dict(),
        "correlation": df.corr().to_dict()
    }
    return eda_report


In [51]:
def plot_correlation(df):
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.corr(), annot=False, cmap='Blues')
    plt.title("Correlation Matrix")
    plt.show()

In [53]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# ---- FIXED PATH ----
data = pd.read_csv('/kaggle/input/ecommerce-sales-data-2023-2024/ecommerce_sales_dataset.csv')

print("Dataset Loaded Successfully!")
print(data.head())

Dataset Loaded Successfully!
  Transaction_ID        Date Customer_ID     Product  Quantity Payment_Method  \
0    TXN_0004163  2023-01-01  CUST_00662      Router         6    Credit Card   
1    TXN_0003251  2023-01-01  CUST_00522  Hard Drive         6    Credit Card   
2    TXN_0001138  2023-01-01  CUST_00941     Charger         3    Credit Card   
3    TXN_0002035  2023-01-01  CUST_00799     Speaker         5      Apple Pay   
4    TXN_0000649  2023-01-01  CUST_00388     Monitor         3     Debit Card   

  Country  Discount_Percentage  Shipping_Cost  Delivery_Days  Customer_Rating  \
0   India                    0            5.0           19.0              5.0   
1     USA                    5           15.0           10.0              5.0   
2     USA                    0           10.0           29.0              5.0   
3     USA                    5            5.0            NaN              4.0   
4     USA                    5           25.0           25.0              NaN  

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


Multi-Agent System

In this project, we implemented a multi-agent system where several specialized agents collaborate to analyze business data. Key agents include:

BI Analyst Agent: Main agent that orchestrates data cleaning, EDA, KPI computation, and insight generation.

Visualization Agent: Generates charts and plots for correlations and trends.

External Data Agent: (Optional) Uses Google Search tool to fetch benchmarks or market data.

Concepts Demonstrated:

Agent powered by an LLM: The BI Analyst Agent is driven by Google’s Gemini LLM to understand natural language queries and orchestrate tasks.

Sequential agents: The workflow is sequential — first clean the data, then analyze it, then generate visualizations, and finally produce recommendations.

Parallel agents (optional): Visualization and KPI computation can run in parallel to optimize performance.

In [54]:
from concurrent.futures import ThreadPoolExecutor, as_completed

data = pd.read_csv('/kaggle/input/ecommerce-sales-data-2023-2024/ecommerce_sales_dataset.csv')
def sales_analysis(df):
    """Analyze total sales and top products."""
    total_sales = df['Sales'].sum()
    top_products = df.groupby('Product')['Sales'].sum().sort_values(ascending=False).head(5)
    return {"total_sales": total_sales, "top_products": top_products}

def customer_segmentation(df):
    """Segment customers by total spending."""
    customer_spending = df.groupby('CustomerID')['Sales'].sum()
    segments = pd.cut(customer_spending, bins=[0, 100, 500, 1000, 10000], labels=['Low', 'Medium', 'High', 'VIP'])
    return segments.value_counts()

def inventory_gap_analysis(df):
    """Find products with low stock."""
    low_stock = df[df['Stock'] < 10].groupby('Product')['Stock'].sum()
    return low_stock

def profitability_analysis(df):
    """Calculate profitability per product."""
    df['Profit'] = df['Sales'] - df['Cost']
    top_profit_products = df.groupby('Product')['Profit'].sum().sort_values(ascending=False).head(5)
    return top_profit_products

# === 3. Run analyses in parallel ===
analysis_functions = [sales_analysis, customer_segmentation, inventory_gap_analysis, profitability_analysis]

results = {}
with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_task = {executor.submit(func, data): func.__name__ for func in analysis_functions}
    for future in as_completed(future_to_task):
        task_name = future_to_task[future]
        try:
            results[task_name] = future.result()
        except Exception as e:
            results[task_name] = f"Error: {e}"

# === 4. Display results ===
for task, result in results.items():
    print(f"\n--- {task} ---")
    print(result)


--- inventory_gap_analysis ---
Error: 'Stock'

--- customer_segmentation ---
Error: 'CustomerID'

--- sales_analysis ---
Error: 'Sales'

--- profitability_analysis ---
Error: 'Sales'


MCP (Multi-Channel Processor)

Orchestrates multiple tasks and tools in parallel

Decides which tool to run based on task and context

Custom Tools

Task-specific tools built for the domain

Examples: sales_analysis, customer_segmentation, inventory_gap, profitability

Built-in Tools

Pre-existing utilities like Google Search or Code Execution

Useful for dynamic problem-solving or additional computations

OpenAPI Tools

Integrate external APIs for real-time data, analytics, or reporting

Examples: e-commerce platform metrics, predictive analytics APIs

Long-Running Operations

Handle heavy computations asynchronously

Support pause/resume, background processing, and checkpointing

Key Idea: Tools work together under MCP orchestration to make the agent autonomous, scalable, and intelligent.

In [59]:
import time
import threading
from typing import Any, Callable, Dict


# ============================================================
# 1. Base Tool Class  (Define FIRST)
# ============================================================
class Tool:
    def __init__(self, name: str, func: Callable):
        self.name = name
        self.func = func

    def call(self, **kwargs):
        return self.func(**kwargs)


# ============================================================
# 2. MCP Tool
# ============================================================
def mcp_process(query: str):
    return {"tool": "MCP", "response": f"MCP processed: {query}"}

mcp_tool = Tool("mcp", mcp_process)


# ============================================================
# 3. Custom Tool
# ============================================================
def custom_discount_tool(price: float, discount: float):
    final_price = price - (price * (discount / 100))
    return {"final_price": final_price}

custom_tool = Tool("discount_tool", custom_discount_tool)


# ============================================================
# 4. Built-in Tools (Google Search + Code Execution)
# ============================================================
def google_search(query: str):
    return {"google_result": f"(Simulated search for): {query}"}

google_tool = Tool("google_search", google_search)


def code_execution(code: str):
    try:
        result = eval(code)
    except Exception as e:
        result = str(e)
    return {"output": result}

code_tool = Tool("code_execution", code_execution)


# ============================================================
# 5. OpenAPI Tool (Simulated)
# ============================================================
def openapi_call(endpoint: str, payload: Dict[str, Any]):
    return {
        "endpoint": endpoint,
        "payload": payload,
        "success": True,
        "message": "Simulated OpenAPI call finished"
    }

openapi_tool = Tool("openapi", openapi_call)


# ============================================================
# 6. Long-Running Task (Pause / Resume)
# ============================================================
class LongTask:
    def __init__(self):
        self.progress = 0
        self.paused = False

    def run(self):
        while self.progress < 100:
            if self.paused:
                print("⏸️ Task paused...")
                time.sleep(1)
                continue

            self.progress += 10
            print(f"⏳ Progress: {self.progress}%")
            time.sleep(1)

        print("✅ Task Completed!")

    def pause(self):
        self.paused = True

    def resume(self):
        print("▶️ Resuming task...")
        self.paused = False


long_task = LongTask()


# ============================================================
# 7. Agent Tool Manager
# ============================================================
class Agent:
    def __init__(self):
        self.tools = {
            "mcp": mcp_tool,
            "custom": custom_tool,
            "google": google_tool,
            "code": code_tool,
            "openapi": openapi_tool,
        }

    def use(self, tool_name: str, **kwargs):
        if tool_name not in self.tools:
            return {"error": "Tool not found"}
        return self.tools[tool_name].call(**kwargs)


# ============================================================
# 8. Example Usage
# ============================================================
agent = Agent()

print("\n🟦 MCP Tool:")
print(agent.use("mcp", query="Process ecommerce data"))

print("\n🟧 Custom Tool (Discount):")
print(agent.use("custom", price=200, discount=15))

print("\n🟩 Google Tool:")
print(agent.use("google", query="Top trending products 2024"))

print("\n🟨 Code Execution Tool:")
print(agent.use("code", code="10 * 10 + 5"))

print("\n🟪 OpenAPI Tool:")
print(agent.use("openapi", endpoint="/products", payload={"limit": 5}))


# ============================================================
# 9. Long Task Demo
# ============================================================
print("\n🟫 Starting long-running task...")
thread = threading.Thread(target=long_task.run)
thread.start()

time.sleep(3)
long_task.pause()

time.sleep(3)
long_task.resume()


🟦 MCP Tool:
{'tool': 'MCP', 'response': 'MCP processed: Process ecommerce data'}

🟧 Custom Tool (Discount):
{'final_price': 170.0}

🟩 Google Tool:
{'google_result': '(Simulated search for): Top trending products 2024'}

🟨 Code Execution Tool:
{'output': 105}

🟪 OpenAPI Tool:
{'endpoint': '/products', 'payload': {'limit': 5}, 'success': True, 'message': 'Simulated OpenAPI call finished'}

🟫 Starting long-running task...
⏳ Progress: 10%
⏳ Progress: 20%
⏳ Progress: 30%
⏸️ Task paused...
⏸️ Task paused...
⏸️ Task paused...
▶️ Resuming task...
⏳ Progress: 40%
⏳ Progress: 50%
⏳ Progress: 60%
⏳ Progress: 70%
⏳ Progress: 80%
⏳ Progress: 90%
⏳ Progress: 100%
✅ Task Completed!


Concept

Sessions and memory enable the BI Analyst Agent to retain context across interactions and over time:

Sessions: Temporary context for a specific user interaction or task. Example: while generating a weekly sales report, the agent remembers filters, date ranges, and selected metrics within that session.

Memory: Persistent knowledge stored across multiple sessions. Example: the agent learns a company’s reporting preferences, commonly used KPIs, or business rules for predictive analytics.

Without sessions and memory, the agent would treat every interaction independently, requiring the user to repeat context and preferences—reducing efficiency and personalization.

In [60]:
import uuid
from typing import Dict, Any


# ============================================================
# 1. In-Memory Session Service (Short-Term State)
# ============================================================
class InMemorySessionService:
    def __init__(self):
        # Stores session data: session_id → {state}
        self.sessions: Dict[str, Dict[str, Any]] = {}

    def create_session(self):
        session_id = str(uuid.uuid4())
        self.sessions[session_id] = {}
        return session_id

    def set(self, session_id: str, key: str, value: Any):
        self.sessions[session_id][key] = value

    def get(self, session_id: str, key: str):
        return self.sessions[session_id].get(key, None)

    def get_all(self, session_id: str):
        return self.sessions[session_id]


# ============================================================
# 2. Long-Term Memory Bank
#    Stores permanent knowledge across sessions
# ============================================================
class MemoryBank:
    def __init__(self):
        # Global memory persists forever
        self.long_term_memory: Dict[str, Any] = {}

    def store(self, key: str, value: Any):
        self.long_term_memory[key] = value

    def recall(self, key: str):
        return self.long_term_memory.get(key, None)

    def all_memory(self):
        return self.long_term_memory


# ============================================================
# 3. Agent Using Sessions + Long-Term Memory
# ============================================================
class Agent:
    def __init__(self, session_service: InMemorySessionService, memory_bank: MemoryBank):
        self.session_service = session_service
        self.memory_bank = memory_bank

    def start_session(self):
        return self.session_service.create_session()

    def remember_in_session(self, session_id: str, key: str, value: Any):
        self.session_service.set(session_id, key, value)
        return {"session_memory_saved": {key: value}}

    def remember_long_term(self, key: str, value: Any):
        self.memory_bank.store(key, value)
        return {"long_term_memory_saved": {key: value}}

    def recall_session(self, session_id: str, key: str):
        return self.session_service.get(session_id, key)

    def recall_long_term(self, key: str):
        return self.memory_bank.recall(key)


# ============================================================
# 4. Demonstration
# ============================================================
session_service = InMemorySessionService()
memory_bank = MemoryBank()
agent = Agent(session_service, memory_bank)

print("\n=== Creating Session ===")
session_id = agent.start_session()
print("New Session ID:", session_id)

print("\n=== Saving Short-Term Session Memory ===")
agent.remember_in_session(session_id, "last_query", "Show me sales data")
agent.remember_in_session(session_id, "user_name", "John Doe")

print("Session State:", session_service.get_all(session_id))

print("\n=== Saving Long-Term Memory ===")
agent.remember_long_term("favorite_category", "Electronics")
agent.remember_long_term("returning_customer", True)

print("Long-Term Memory:", memory_bank.all_memory())

print("\n=== Recall Short-Term Memory ===")
print("Last query:", agent.recall_session(session_id, "last_query"))

print("\n=== Recall Long-Term Memory ===")


=== Creating Session ===
New Session ID: af544e6c-bfce-4463-81c4-b581ee453b7f

=== Saving Short-Term Session Memory ===
Session State: {'last_query': 'Show me sales data', 'user_name': 'John Doe'}

=== Saving Long-Term Memory ===
Long-Term Memory: {'favorite_category': 'Electronics', 'returning_customer': True}

=== Recall Short-Term Memory ===
Last query: Show me sales data

=== Recall Long-Term Memory ===


In [61]:
import uuid
from typing import List, Dict, Any


# ============================================================
# 1. Short-Term Session Memory
# ============================================================
class SessionMemory:
    def __init__(self):
        self.sessions: Dict[str, List[Dict[str, str]]] = {}

    def create(self):
        sid = str(uuid.uuid4())
        self.sessions[sid] = []
        return sid

    def add_message(self, session_id: str, role: str, content: str):
        self.sessions[session_id].append({"role": role, "content": content})

    def get_history(self, session_id: str):
        return self.sessions[session_id]


# ============================================================
# 2. Long-Term Memory (facts + user preferences)
# ============================================================
class LongTermMemory:
    def __init__(self):
        self.facts: Dict[str, Any] = {}

    def store(self, key: str, value: Any):
        self.facts[key] = value

    def retrieve(self, key: str):
        return self.facts.get(key, None)

    def all(self):
        return self.facts


# ============================================================
# 3. Context Compaction Engine
# ============================================================
class ContextCompactor:
    def compact(self, messages: List[Dict[str, str]], max_messages: int = 5):
        """
        Purpose:
        - Reduce long history.
        - Keep only key information.
        """
        if len(messages) <= max_messages:
            return messages  # no compaction needed

        # Extract the "core" of the conversation
        important_points = []
        for msg in messages:
            text = msg["content"].lower()

            # Heuristic rules for what to keep
            if any(
                keyword in text
                for keyword in ["sales", "product", "price", "dataset", "analysis", "customer"]
            ):
                important_points.append(msg)

        # If no important messages detected, fallback to newest messages
        if not important_points:
            important_points = messages[-max_messages:]

        # Convert to compacted form
        summary = [
            {"role": "system", "content": "Context summary: Key points retained."}
        ] + important_points[-max_messages:]

        return summary


# ============================================================
# 4. Context Builder (sessions + compaction + memory)
# ============================================================
class ContextBuilder:
    def __init__(self, session_memory: SessionMemory, long_memory: LongTermMemory, compactor: ContextCompactor):
        self.session_memory = session_memory
        self.long_memory = long_memory
        self.compactor = compactor

    def build_context(self, session_id: str):
        raw_history = self.session_memory.get_history(session_id)

        # compact the history
        compact_history = self.compactor.compact(raw_history)

        # insert long-term memory facts on top
        long_memory_data = [
            {"role": "system", "content": f"Long-term memory: {key} → {value}"}
            for key, value in self.long_memory.all().items()
        ]

        final_context = long_memory_data + compact_history
        return final_context


# ============================================================
# 5. Demo Agent
# ============================================================
class Agent:
    def __init__(self, context_builder: ContextBuilder, session_memory: SessionMemory):
        self.context_builder = context_builder
        self.session_memory = session_memory

    def send(self, session_id: str, user_message: str):
        # save message
        self.session_memory.add_message(session_id, "user", user_message)

        # build context automatically
        context = self.context_builder.build_context(session_id)

        # (Simulated) agent response
        response = f"(Agent using {len(context)} context items) → Got it: {user_message}"

        # save response
        self.session_memory.add_message(session_id, "assistant", response)

        return response, context


# ============================================================
# 6. Run Example
# ============================================================
session_mem = SessionMemory()
long_mem = LongTermMemory()
compactor = ContextCompactor()
builder = ContextBuilder(session_mem, long_mem, compactor)
agent = Agent(builder, session_mem)

# Create session
sid = session_mem.create()

# Add long-term memory
long_mem.store("favorite_category", "Electronics")
long_mem.store("preferred_format", "Short answers")

# Simulate conversation
agent.send(sid, "Analyze ecommerce sales dataset")
agent.send(sid, "Which product had the highest revenue last month?")
agent.send(sid, "Show me customer segmentation")
agent.send(sid, "I want to predict future sales using ML")
agent.send(sid, "Plot monthly revenue trend")

# Now trigger compaction
response, context = agent.send(sid, "Give me inventory gap analysis")

print("\n=== Agent Response ===")
print(response)

print("\n=== Final Compacted Context ===")
for c in context:
    print(f"{c['role'].upper()}: {c['content']}")


=== Agent Response ===
(Agent using 8 context items) → Got it: Give me inventory gap analysis

=== Final Compacted Context ===
SYSTEM: Long-term memory: favorite_category → Electronics
SYSTEM: Long-term memory: preferred_format → Short answers
SYSTEM: Context summary: Key points retained.
USER: Show me customer segmentation
ASSISTANT: (Agent using 7 context items) → Got it: Show me customer segmentation
USER: I want to predict future sales using ML
ASSISTANT: (Agent using 8 context items) → Got it: I want to predict future sales using ML
USER: Give me inventory gap analysis


In [62]:
import time
import uuid
from typing import Any, Dict, List
import logging

# ============================================================
# 1. Setup Logging
# ============================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%H:%M:%S'
)

logger = logging.getLogger("AgentLogger")

# ============================================================
# 2. Metrics Collector
# ============================================================
class MetricsCollector:
    def __init__(self):
        self.metrics = {
            "messages_processed": 0,
            "sessions_created": 0,
            "errors": 0,
        }

    def increment(self, key: str, amount: int = 1):
        if key in self.metrics:
            self.metrics[key] += amount

    def report(self):
        return self.metrics.copy()


# ============================================================
# 3. Simple Tracing Decorator
# ============================================================
def trace(func):
    def wrapper(*args, **kwargs):
        trace_id = str(uuid.uuid4())[:8]
        logger.info(f"[Trace {trace_id}] START {func.__name__}")
        start_time = time.time()
        try:
            result = func(*args, **kwargs)
            duration = time.time() - start_time
            logger.info(f"[Trace {trace_id}] END {func.__name__} (took {duration:.3f}s)")
            return result
        except Exception as e:
            duration = time.time() - start_time
            logger.error(f"[Trace {trace_id}] ERROR in {func.__name__}: {e} (took {duration:.3f}s)")
            raise
    return wrapper


# ============================================================
# 4. Session & Memory (Simplified)
# ============================================================
class SessionService:
    def __init__(self):
        self.sessions: Dict[str, List[str]] = {}

    @trace
    def create_session(self):
        session_id = str(uuid.uuid4())
        self.sessions[session_id] = []
        return session_id

    @trace
    def add_message(self, session_id: str, message: str):
        if session_id in self.sessions:
            self.sessions[session_id].append(message)


class MemoryBank:
    def __init__(self):
        self.memory: Dict[str, Any] = {}

    @trace
    def store(self, key: str, value: Any):
        self.memory[key] = value

    @trace
    def retrieve(self, key: str):
        return self.memory.get(key)


# ============================================================
# 5. Agent with Observability
# ============================================================
class ObservantAgent:
    def __init__(self, session_service: SessionService, memory_bank: MemoryBank, metrics: MetricsCollector):
        self.session_service = session_service
        self.memory_bank = memory_bank
        self.metrics = metrics

    @trace
    def process_message(self, session_id: str, message: str):
        logger.info(f"Processing message in session {session_id}: {message}")
        self.session_service.add_message(session_id, message)
        self.metrics.increment("messages_processed")

        # Simulate processing and storing key info in memory
        if "favorite" in message.lower():
            self.memory_bank.store("favorite_item", message)
        
        response = f"(Agent response) Received: {message}"
        return response


# ============================================================
# 6. Demo Run
# ============================================================
metrics = MetricsCollector()
sessions = SessionService()
memory = MemoryBank()
agent = ObservantAgent(sessions, memory, metrics)

# Create session
session_id = sessions.create_session()
metrics.increment("sessions_created")
logger.info(f"New session created: {session_id}")

# Process messages
msgs = [
    "Analyze ecommerce sales data",
    "User's favorite product is Laptop",
    "Predict future sales trends"
]

for m in msgs:
    resp = agent.process_message(session_id, m)
    logger.info(f"Agent response: {resp}")

# Report metrics
logger.info(f"Metrics: {metrics.report()}")

# Access memory
logger.info(f"Memory contents: {memory.memory}")

# Access session history
logger.info(f"Session history: {sessions.sessions[session_id]}")

In [63]:
import logging
from typing import List, Dict

# ============================================================
# 1. Setup Logging
# ============================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger("AgentEvaluation")

# ============================================================
# 2. Evaluation Metrics
# ============================================================
class EvaluationMetrics:
    def __init__(self):
        self.metrics = {
            "tasks_attempted": 0,
            "tasks_succeeded": 0,
            "tasks_failed": 0,
            "average_response_quality": 0.0,
        }
        self.response_scores: List[float] = []

    def record_task(self, success: bool, score: float = None):
        self.metrics["tasks_attempted"] += 1
        if success:
            self.metrics["tasks_succeeded"] += 1
        else:
            self.metrics["tasks_failed"] += 1
        if score is not None:
            self.response_scores.append(score)
            self.metrics["average_response_quality"] = sum(self.response_scores) / len(self.response_scores)

    def report(self):
        return self.metrics.copy()


# ============================================================
# 3. Simulated Agent (for demo)
# ============================================================
class DemoAgent:
    def __init__(self):
        self.tools = ["sales_analysis", "customer_segmentation", "inventory_gap", "profitability"]

    def run_task(self, task_name: str):
        # Simulate task execution
        if task_name not in self.tools:
            return False, 0.0  # failed
        # Simulate a quality score 0.0-1.0
        import random
        success = random.random() > 0.1  # 90% chance of success
        score = random.uniform(0.7, 1.0) if success else random.uniform(0.0, 0.6)
        return success, score


# ============================================================
# 4. Agent Evaluator
# ============================================================
class AgentEvaluator:
    def __init__(self, agent, metrics: EvaluationMetrics):
        self.agent = agent
        self.metrics = metrics

    def evaluate_tasks(self, task_list: List[str]):
        for task in task_list:
            success, score = self.agent.run_task(task)
            self.metrics.record_task(success, score)
            logger.info(f"Task '{task}' success={success}, score={score:.2f}")

    def final_report(self):
        report = self.metrics.report()
        logger.info("=== Agent Evaluation Report ===")
        logger.info(report)
        return report


# ============================================================
# 5. Demo Run
# ============================================================
agent = DemoAgent()
metrics = EvaluationMetrics()
evaluator = AgentEvaluator(agent, metrics)

tasks_to_test = [
    "sales_analysis",
    "customer_segmentation",
    "inventory_gap",
    "profitability",
    "nonexistent_tool"  # intentional failure
]

# Run evaluation
evaluator.evaluate_tasks(tasks_to_test)

# Report results
report = evaluator.final_report()
print(report)

{'tasks_attempted': 5, 'tasks_succeeded': 3, 'tasks_failed': 2, 'average_response_quality': 0.6202124761155842}


In [64]:
import uuid
import queue
import threading
import time
from typing import Dict, Any

# ============================================================
# 1. Message Definition
# ============================================================
class A2AMessage:
    def __init__(self, sender: str, receiver: str, task: str, data: Dict[str, Any]):
        self.sender = sender
        self.receiver = receiver
        self.task = task
        self.data = data
        self.message_id = str(uuid.uuid4())
        self.timestamp = time.time()

    def to_dict(self):
        return {
            "sender": self.sender,
            "receiver": self.receiver,
            "task": self.task,
            "data": self.data,
            "message_id": self.message_id,
            "timestamp": self.timestamp,
        }


# ============================================================
# 2. Agent Base Class
# ============================================================
class Agent:
    def __init__(self, name: str):
        self.name = name
        self.inbox = queue.Queue()

    def send(self, receiver: "Agent", task: str, data: Dict[str, Any]):
        msg = A2AMessage(self.name, receiver.name, task, data)
        receiver.inbox.put(msg)
        print(f"[{self.name}] Sent task '{task}' to {receiver.name} (msg_id={msg.message_id})")

    def process_messages(self):
        while True:
            try:
                msg: A2AMessage = self.inbox.get(timeout=1)
                print(f"[{self.name}] Received task '{msg.task}' from {msg.sender}")
                self.handle_task(msg)
            except queue.Empty:
                break

    def handle_task(self, msg: A2AMessage):
        # Override this in subclasses
        print(f"[{self.name}] Handling task: {msg.task} with data {msg.data}")


# ============================================================
# 3. Specialized Agent
# ============================================================
class SalesAgent(Agent):
    def handle_task(self, msg: A2AMessage):
        if msg.task == "analyze_sales":
            print(f"[{self.name}] Analyzing sales data: {msg.data}")
            # Simulate analysis
            time.sleep(0.5)
            print(f"[{self.name}] Analysis complete for {msg.sender}")
        else:
            print(f"[{self.name}] Unknown task: {msg.task}")


# ============================================================
# 4. Demo A2A Communication
# ============================================================
agent_a = Agent("Agent_A")
agent_b = SalesAgent("SalesAgent_B")

# Agent A sends task to B
agent_a.send(agent_b, task="analyze_sales", data={"dataset": "ecommerce_2023.csv"})

# Process messages
thread_b = threading.Thread(target=agent_b.process_messages)
thread_b.start()
thread_b.join()

print("\n✅ A2A demo complete")

[Agent_A] Sent task 'analyze_sales' to SalesAgent_B (msg_id=2e6cc45b-b832-4efd-b553-28a8b57e4189)
[SalesAgent_B] Received task 'analyze_sales' from Agent_A
[SalesAgent_B] Analyzing sales data: {'dataset': 'ecommerce_2023.csv'}
[SalesAgent_B] Analysis complete for Agent_A

✅ A2A demo complete


In [65]:
from flask import Flask, request, jsonify
import uuid
import threading
import time
import logging

# -----------------------------
# Logging
# -----------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("DeployedAgent")

# -----------------------------
# Memory & Sessions
# -----------------------------
class SessionService:
    def __init__(self):
        self.sessions = {}

    def create_session(self):
        session_id = str(uuid.uuid4())
        self.sessions[session_id] = []
        return session_id

    def add_message(self, session_id, message):
        self.sessions[session_id].append(message)

    def get_history(self, session_id):
        return self.sessions.get(session_id, [])

session_service = SessionService()

# -----------------------------
# Simple Agent
# -----------------------------
class Agent:
    def __init__(self, session_service):
        self.session_service = session_service

    def respond(self, session_id, user_message):
        self.session_service.add_message(session_id, {"role": "user", "content": user_message})
        # Here you could integrate memory, tools, context, A2A calls, etc.
        response = f"(Agent response) You said: {user_message}"
        self.session_service.add_message(session_id, {"role": "agent", "content": response})
        return response

agent = Agent(session_service)

# -----------------------------
# Flask API
# -----------------------------
app = Flask(__name__)

@app.route("/create_session", methods=["POST"])
def create_session():
    session_id = session_service.create_session()
    logger.info(f"New session created: {session_id}")
    return jsonify({"session_id": session_id})

@app.route("/send_message", methods=["POST"])
def send_message():
    data = request.json
    session_id = data.get("session_id")
    message = data.get("message")
    if not session_id or not message:
        return jsonify({"error": "session_id and message are required"}), 400

    response = agent.respond(session_id, message)
    history = session_service.get_history(session_id)
    return jsonify({"response": response, "session_history": history})

# -----------------------------
# Run server (in Kaggle use debug=True)
# -----------------------------
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.19.2.2:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 991

SystemExit: 1

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
